### MobileNet
- Neural network which is around 28MB. 
- Runs on mobile devices.
- Single-shot detection.

### Bring in an image to this model (do inference on a trained model)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions

In [2]:
 model = MobileNet() # weights="path to the h5 file"

In [3]:
img = image.load_img('./data/hotdog/train/hot_dog/1017226.jpg', target_size=(224,224))
img_array = image.img_to_array(img) # Converting from PIL object into a numpy array
img_array.shape

(224, 224, 3)

In [4]:
probas = model.predict(img_array.reshape(1, 224,224,3))

In [5]:
decode_predictions(probas)

[[('n02782093', 'balloon', 0.4686488),
  ('n09256479', 'coral_reef', 0.10136139),
  ('n02606052', 'rock_beauty', 0.08158921),
  ('n04209239', 'shower_curtain', 0.0690787),
  ('n03530642', 'honeycomb', 0.031012459)]]

#### Load as a json file (if no access to internet)

In [6]:
#import json
#with open("json file path") as f:
#    imagenet = json.load(f)
# probas

## Transfer learning

- Bring back `MobileNet` and do not include the top layer on the network (the last layer excluded).

In [7]:
base_model = MobileNet()

#### Take the output of the last but one layer, and use it to build on top of it.

In [8]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model, Sequential

In [9]:
len(base_model.layers)

93

In [10]:
new_model = Sequential([base_model.layers[i] for i in range(92)])
new_model.trainable = False  # True = We do fine-tuning, False = Feature extraction + fine-tuning comes later
new_model.add(Dense(512, activation='relu'))
new_model.add(Dense(2, activation='softmax'))

In [11]:
#new_new_model = Sequential(base_model.layers[::-1]+[Dense(512, activation='relu'), Dense(2, activation='softmax')])

#### Use generator to bring in the training images for hotdog/not hotdog 

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input
                                   , horizontal_flip=True, vertical_flip=True)

In [14]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input
                                   , horizontal_flip=True, vertical_flip=True)

In [15]:
train_generator = train_datagen.flow_from_directory('./data/hotdog/train/', 
                                                    target_size=(224,224), 
                                                    color_mode='rgb', 
                                                    class_mode='categorical', 
                                                    batch_size=16)

Found 498 images belonging to 2 classes.


In [16]:
test_generator = test_datagen.flow_from_directory('./data/hotdog/test/', 
                                                    target_size=(224,224), 
                                                    color_mode='rgb', 
                                                    class_mode='categorical', 
                                                    batch_size=16)

Found 500 images belonging to 2 classes.


In [17]:
new_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
history = new_model.fit_generator(generator=train_generator,
                                  epochs=20,
                                  validation_data=test_generator
                                 )

Epoch 1/20
32/32 [==============================] - 171s 5s/step - loss: 2.0991 - accuracy: 0.7189 - val_loss: 4.9146 - val_accuracy: 0.5620
Epoch 2/20
32/32 [==============================] - 174s 5s/step - loss: 0.7475 - accuracy: 0.8534 - val_loss: 5.1640 - val_accuracy: 0.5700
Epoch 3/20
32/32 [==============================] - 174s 5s/step - loss: 0.5712 - accuracy: 0.8815 - val_loss: 5.0946 - val_accuracy: 0.5440
Epoch 4/20
32/32 [==============================] - 174s 5s/step - loss: 0.4022 - accuracy: 0.9096 - val_loss: 1.1729 - val_accuracy: 0.8060
Epoch 5/20
32/32 [==============================] - 173s 5s/step - loss: 0.4036 - accuracy: 0.9036 - val_loss: 1.2991 - val_accuracy: 0.7780
Epoch 6/20
32/32 [==============================] - 173s 5s/step - loss: 0.1845 - accuracy: 0.9438 - val_loss: 1.7937 - val_accuracy: 0.7340
Epoch 7/20
32/32 [==============================] - 173s 5s/step - loss: 0.2644 - accuracy: 0.9317 - val_loss: 2.1613 - val_accuracy: 0.6900
Epoch 8/20
32

In [19]:
def load_img(file):
    img = image.load_img(file, target_size=(224,224))
    img_array = image.img_to_array(img) # Converting from PIL object into a numpy array
    return img_array.reshape(1, 224,224,3)

In [20]:
preds_to_labels = { train_generator.class_indices[k]:k for k in train_generator.class_indices.keys()}

In [21]:
counter = {'hot_dog':0, 'not_hot_dog':0}
import glob
for file in glob.glob('./data/hotdog/test/hot_dog/*'):
    preds = new_model.predict(load_img(file))
    pred = np.argmax(preds)
    counter[preds_to_labels[pred]] += 1

In [22]:
counter

{'hot_dog': 94, 'not_hot_dog': 156}

## Model persistance

In [23]:
import tensorflow as tf

In [24]:
tf.saved_model.save(new_model, "./tmp/mobilenet/1/")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./tmp/mobilenet/1/assets
